In [0]:
%pip install openpyxl 
%pip install regex
%pip install tqdm

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/57.7 kB ? eta -:--:--
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/78.5 kB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 3.7 MB/s eta 0:00:00
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
import pandas as pd
import os
import json
from dataclasses import dataclass
import requests
from typing import Dict, List, Optional, Any
import time
from tqdm import tqdm

#Nouveau regex 

In [0]:
@dataclass
class AuthConfig:
    base_url: str
    client_id: str
    client_secret: str
    scope: str

class AuthClient:
    def __init__(self, config: AuthConfig):
        self.config = config

    def get_token(self) -> Dict[str, str]:
        try:
            response = requests.post(
                f"{self.config.base_url}/as/token.oauth2",
                headers={"Content-Type": "application/x-www-form-urlencoded"},
                data={
                    "grant_type": "client_credentials",
                    "client_id": self.config.client_id,
                    "client_secret": self.config.client_secret,
                    "scope": self.config.scope
                }
            )
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException as e:
            raise Exception(f"Échec de l'authentification: {str(e)}")

def get_auth_token() -> str:
    config = AuthConfig(
        base_url="https://onelogin.stg.axa.com",
        client_id="VJKkswZb",
        client_secret="yG3pl6nkYhvJlQVD1y7xbZBl692YRytdpUJeODKg5BAutoEgxhkqzXLFYGqH5zlp",
        scope="urn:grp:chatgpt"
    )
    client = AuthClient(config)
    result = client.get_token()
    return result.get("access_token", "")

class SecureGPTRegexImprover:
    def __init__(self, 
                token: str,
                temperature: float = 0.05,
                base_url: str = "https://api-pp.se.axa-go.axa.com/ago-m365-securegpt-bapi-v1-vrs",
                model: str = "gpt-4o-2024-08-06"):
        self.token = token
        self.temperature = temperature
        self.base_url = base_url
        self.model = model
        self.costs = {
            'input_token': 2.50/1_000_000,
            'output_token': 10.00/1_000_000
        }

    def calculate_cost(self, response: Dict) -> Dict[str, float]:
        metrics = {'token_cost': 0, 'total_cost': 0}

        if 'usage' in response:
            input_tokens = response['usage'].get('prompt_tokens', 0)
            output_tokens = response['usage'].get('completion_tokens', 0)
            metrics['token_cost'] = (input_tokens * self.costs['input_token'] + 
                                   output_tokens * self.costs['output_token'])
            metrics['total_cost'] = metrics['token_cost']

        return metrics

    def chat(self, messages: List[Dict[str, Any]], **kwargs) -> Dict:
        url = f"{self.base_url}/deployments/{self.model}/chat/completions"
        headers = {
            "Content-Type": "application/json",
            "Authorization": f"Bearer {self.token}"
        }
        data = {
            "messages": messages,
            "temperature": self.temperature,
            "max_tokens": kwargs.get("max_tokens", 4000)
        }

        try:
            response = requests.post(url, headers=headers, json=data)
            response.raise_for_status()
            api_response = response.json()
            cost_metrics = self.calculate_cost(api_response)
            api_response['cost_metrics'] = cost_metrics
            return api_response
        except requests.exceptions.RequestException as e:
            print(f"Détails de l'erreur API: {response.text if 'response' in locals() else 'Pas de réponse'}")
            raise Exception(f"Échec de l'appel API: {str(e)}")

    def improve_regex_batch(self, 
                           batch_data: List[Dict[str, str]], 
                           system_prompt: str,
                           additional_user_info: str = "") -> List[Dict]:
        formatted_batch = json.dumps(batch_data, ensure_ascii=False, indent=2)
        user_content = f"""
Voici un lot de règles regex à améliorer:

{formatted_batch}

{additional_user_info}

Pour chaque regex, suggérez une version améliorée qui prend en compte:
- Variations de casse (majuscules/minuscules)
- Variations grammaticales (singulier/pluriel, masculin/féminin)
- Formats alternatifs (pour les dates, nombres, etc.)
- Robustesse face aux erreurs courantes

Renvoyez-moi un JSON avec les mêmes clés que l'entrée, mais avec les colonnes "regex_rule_amelio" et "amelio_explanation" remplies.
"""
        
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_content}
        ]
        
        max_retries = 5
        retry_delay = 15
        
        for attempt in range(max_retries):
            try:
                response = self.chat(messages)
                response_content = response['choices'][0]['message']['content']
                
                clean_content = response_content.strip()
                if clean_content.startswith("```json"):
                    clean_content = clean_content[7:]
                if clean_content.endswith("```"):
                    clean_content = clean_content[:-3]
                clean_content = clean_content.strip()
                
                improved_batch = json.loads(clean_content)
                
                for item in improved_batch:
                    item['cost_metrics'] = response.get('cost_metrics', {})
                
                return improved_batch
            
            except Exception as e:
                if attempt < max_retries - 1:
                    print(f"Erreur lors de l'amélioration du lot (tentative {attempt+1}/{max_retries}): {str(e)}")
                    print(f"Nouvelle tentative dans {retry_delay} secondes...")
                    time.sleep(retry_delay)
                else:
                    print(f"Échec de l'amélioration du lot après {max_retries} tentatives: {str(e)}")
                    for item in batch_data:
                        item['regex_rule_amelio'] = ""
                        item['amelio_explanation'] = ""
                        item['cost_metrics'] = {'ERROR': str(e)}
                    return batch_data

In [0]:
def read_regex_files(excel_file: str) -> tuple:
    try:
        regex_mapping = pd.read_excel(excel_file, sheet_name='regex_mapping')
        regex_definition = pd.read_excel(excel_file, sheet_name='regex_definition')
        
        print(f"Onglet regex_mapping chargé avec {len(regex_mapping)} lignes")
        print(f"Onglet regex_definition chargé avec {len(regex_definition)} lignes")
        
        return regex_mapping, regex_definition
    except Exception as e:
        raise ValueError(f"Erreur lors de la lecture du fichier Excel: {str(e)}")

def join_regex_dataframes(regex_mapping: pd.DataFrame, regex_definition: pd.DataFrame) -> pd.DataFrame:
    try:
        if 'regex_id' not in regex_mapping.columns:
            raise ValueError("La colonne 'regex_id' est manquante dans regex_mapping")
        if 'regex_id' not in regex_definition.columns:
            raise ValueError("La colonne 'regex_id' est manquante dans regex_definition")
            
        joined_df = pd.merge(
            regex_mapping,
            regex_definition,
            on='regex_id',
            how='inner'
        )
        
        print(f"Jointure effectuée avec succès, résultat: {len(joined_df)} lignes")
        return joined_df
    except Exception as e:
        raise ValueError(f"Erreur lors de la jointure des dataframes: {str(e)}")


In [0]:
import pandas as pd
import os
import json
from dataclasses import dataclass
import requests
from typing import Dict, List, Optional, Any
import time
from tqdm import tqdm

@dataclass
class AuthConfig:
    base_url: str
    client_id: str
    client_secret: str
    scope: str

class AuthClient:
    def __init__(self, config: AuthConfig):
        self.config = config

    def get_token(self) -> Dict[str, str]:
        try:
            response = requests.post(
                f"{self.config.base_url}/as/token.oauth2",
                headers={"Content-Type": "application/x-www-form-urlencoded"},
                data={
                    "grant_type": "client_credentials",
                    "client_id": self.config.client_id,
                    "client_secret": self.config.client_secret,
                    "scope": self.config.scope
                }
            )
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException as e:
            raise Exception(f"Échec de l'authentification: {str(e)}")

def get_auth_token() -> str:
    config = AuthConfig(
        base_url="https://onelogin.stg.axa.com",
        client_id="VJKkswZb",
        client_secret="yG3pl6nkYhvJlQVD1y7xbZBl692YRytdpUJeODKg5BAutoEgxhkqzXLFYGqH5zlp",
        scope="urn:grp:chatgpt"
    )
    client = AuthClient(config)
    result = client.get_token()
    return result.get("access_token", "")

class SecureGPTRegexImprover:
    def __init__(self, 
                token: str,
                temperature: float = 0.05,
                base_url: str = "https://api-pp.se.axa-go.axa.com/ago-m365-securegpt-bapi-v1-vrs",
                model: str = "gpt-4o-2024-08-06"):
        self.token = token
        self.temperature = temperature
        self.base_url = base_url
        self.model = model
        self.costs = {
            'input_token': 2.50/1_000_000,
            'output_token': 10.00/1_000_000
        }

    def calculate_cost(self, response: Dict) -> Dict[str, float]:
        metrics = {'token_cost': 0, 'total_cost': 0}

        if 'usage' in response:
            input_tokens = response['usage'].get('prompt_tokens', 0)
            output_tokens = response['usage'].get('completion_tokens', 0)
            metrics['token_cost'] = (input_tokens * self.costs['input_token'] + 
                                   output_tokens * self.costs['output_token'])
            metrics['total_cost'] = metrics['token_cost']

        return metrics

    def chat(self, messages: List[Dict[str, Any]], **kwargs) -> Dict:
        url = f"{self.base_url}/deployments/{self.model}/chat/completions"
        headers = {
            "Content-Type": "application/json",
            "Authorization": f"Bearer {self.token}"
        }
        data = {
            "messages": messages,
            "temperature": self.temperature,
            "max_tokens": kwargs.get("max_tokens", 4000)
        }

        try:
            response = requests.post(url, headers=headers, json=data)
            response.raise_for_status()
            api_response = response.json()
            cost_metrics = self.calculate_cost(api_response)
            api_response['cost_metrics'] = cost_metrics
            return api_response
        except requests.exceptions.RequestException as e:
            print(f"Détails de l'erreur API: {response.text if 'response' in locals() else 'Pas de réponse'}")
            raise Exception(f"Échec de l'appel API: {str(e)}")

    def improve_regex_batch(self, 
                           batch_data: List[Dict[str, str]], 
                           system_prompt: str,
                           additional_user_info: str = "") -> List[Dict]:
        formatted_batch = json.dumps(batch_data, ensure_ascii=False, indent=2)
        user_content = f"""
Voici un lot de règles regex à améliorer:

{formatted_batch}

{additional_user_info}

Pour chaque regex, suggérez une version améliorée qui prend en compte:
- Variations de casse (majuscules/minuscules)
- Variations grammaticales (singulier/pluriel, masculin/féminin)
- Formats alternatifs (pour les dates, nombres, etc.)
- Robustesse face aux erreurs courantes

Renvoyez-moi un JSON avec les mêmes clés que l'entrée, mais avec les colonnes "regex_rule_amelio" et "amelio_explanation" remplies.
"""
        
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_content}
        ]
        
        max_retries = 5
        retry_delay = 15
        
        for attempt in range(max_retries):
            try:
                response = self.chat(messages)
                response_content = response['choices'][0]['message']['content']
                
                clean_content = response_content.strip()
                if clean_content.startswith("```json"):
                    clean_content = clean_content[7:]
                if clean_content.endswith("```"):
                    clean_content = clean_content[:-3]
                clean_content = clean_content.strip()
                
                improved_batch = json.loads(clean_content)
                
                for item in improved_batch:
                    item['cost_metrics'] = response.get('cost_metrics', {})
                
                return improved_batch
            
            except Exception as e:
                if attempt < max_retries - 1:
                    print(f"Erreur lors de l'amélioration du lot (tentative {attempt+1}/{max_retries}): {str(e)}")
                    print(f"Nouvelle tentative dans {retry_delay} secondes...")
                    time.sleep(retry_delay)
                else:
                    print(f"Échec de l'amélioration du lot après {max_retries} tentatives: {str(e)}")
                    # Si l'API échoue après toutes les tentatives, indiquer clairement l'échec dans les colonnes résultats
                    for item in batch_data:
                        item['regex_rule_amelio'] = "ERREUR API: Échec après 5 tentatives"
                        item['amelio_explanation'] = f"Erreur: {str(e)}"
                        item['cost_metrics'] = {'ERROR': str(e)}
                    return batch_data

def read_regex_files(excel_file: str) -> tuple:
    try:
        # Modification pour lire le format spécifique de l'onglet regex_mapping
        # Dans ce format, la première ligne contient les noms de colonnes et la seconde ligne contient les IDs
        raw_mapping = pd.read_excel(excel_file, sheet_name='regex_mapping', header=None)
        
        # La première ligne contient les noms de colonnes
        column_names = raw_mapping.iloc[0].tolist()
        
        # La deuxième ligne contient les IDs
        regex_ids = raw_mapping.iloc[1].tolist()
        
        # Créer un DataFrame transformé au format attendu
        mapping_data = []
        for i, (column_name, regex_id) in enumerate(zip(column_names, regex_ids)):
            if pd.notna(column_name) and pd.notna(regex_id):
                mapping_data.append({
                    'column_name': column_name,
                    'regex_id': regex_id
                })
        
        regex_mapping = pd.DataFrame(mapping_data)
        
        # Lire l'onglet regex_definition normalement
        regex_definition = pd.read_excel(excel_file, sheet_name='regex_definition')
        
        print(f"Onglet regex_mapping transformé avec {len(regex_mapping)} lignes")
        print(f"Onglet regex_definition chargé avec {len(regex_definition)} lignes")
        
        return regex_mapping, regex_definition
    except Exception as e:
        raise ValueError(f"Erreur lors de la lecture du fichier Excel: {str(e)}")

def join_regex_dataframes(regex_mapping: pd.DataFrame, regex_definition: pd.DataFrame) -> pd.DataFrame:
    try:
        if 'regex_id' not in regex_mapping.columns:
            raise ValueError("La colonne 'regex_id' est manquante dans regex_mapping")
        if 'regex_id' not in regex_definition.columns:
            raise ValueError("La colonne 'regex_id' est manquante dans regex_definition")
            
        joined_df = pd.merge(
            regex_mapping,
            regex_definition,
            on='regex_id',
            how='inner'
        )
        
        print(f"Jointure effectuée avec succès, résultat: {len(joined_df)} lignes")
        return joined_df
    except Exception as e:
        raise ValueError(f"Erreur lors de la jointure des dataframes: {str(e)}")

def process_regex_improvement(
    excel_file: str,
    output_file: str,
    system_prompt: str,
    additional_user_info: str = "",
    batch_size: int = 5,
    save_interval: int = 10
) -> None:
    regex_mapping, regex_definition = read_regex_files(excel_file)
    
    joined_df = join_regex_dataframes(regex_mapping, regex_definition)
    
    improver = SecureGPTRegexImprover(
        token=get_auth_token(),
        temperature=0.1
    )
    
    data_to_improve = []
    for _, row in joined_df.iterrows():
        item = {
            'regex_id': row['regex_id'],
            'column_name': row['column_name'] if 'column_name' in row else "",
            'regex_rule': row['regex_rule'] if 'regex_rule' in row else "",
            'regex_description': row['regex_description'] if 'regex_description' in row else "",
            'regex_rule_amelio': "",
            'amelio_explanation': ""
        }
        data_to_improve.append(item)
    
    results = []
    total_cost = 0
    
    for i in tqdm(range(0, len(data_to_improve), batch_size), desc="Traitement des lots"):
        batch = data_to_improve[i:i+batch_size]
        
        try:
            improved_batch = improver.improve_regex_batch(
                batch, 
                system_prompt, 
                additional_user_info
            )
            
            results.extend(improved_batch)
            
            for item in improved_batch:
                if 'cost_metrics' in item and 'total_cost' in item['cost_metrics']:
                    total_cost += item['cost_metrics']['total_cost']
            
            if (i // batch_size) % save_interval == 0:
                temp_df = pd.DataFrame(results)
                temp_output = f"{os.path.splitext(output_file)[0]}_temp_{i}.xlsx"
                temp_df.to_excel(temp_output, index=False)
                print(f"Résultats intermédiaires sauvegardés dans {temp_output}")
                print(f"Coût cumulatif: {total_cost:.6f} $")
            
            time.sleep(1)
            
        except Exception as e:
            print(f"Erreur lors du traitement du lot {i//batch_size + 1}: {str(e)}")
    
    results_df = pd.DataFrame(results)
    results_df.to_excel(output_file, index=False)
    
    print(f"Traitement terminé. Résultats sauvegardés dans {output_file}")
    print(f"Coût total: {total_cost:.6f} $")

if __name__ == "__main__":
    EXCEL_FILE = "Regex_Health_Listado_asegurados.xlsx"
    OUTPUT_FILE = "regex_improved_results.xlsx"
    
    SYSTEM_PROMPT = """Vous êtes un expert en expressions régulières (regex).
Votre tâche est d'améliorer les regex pour les rendre plus robustes et complètes.

Règles importantes:
1. Assurez-vous que les regex améliorées couvrent plus de cas d'utilisation tout en restant précises.
2. Prenez en compte les variations possibles comme la casse, le pluriel/singulier, les formats alternatifs, etc.
3. Fournissez une explication claire et EN FRANÇAIS de vos modifications et des nouveaux cas couverts.
4. Assurez-vous que les regex améliorées restent compatibles avec le format Python/PCRE.
5. Testez mentalement vos regex sur divers exemples pour garantir leur robustesse.
6. N'hésitez pas à utiliser des groupes non capturants (?:) pour améliorer la performance.

Votre sortie doit être un JSON respectant strictement le format d'entrée, avec les champs "regex_rule_amelio" et "amelio_explanation" remplis. La colonne "amelio_explanation" DOIT être rédigée en français."""
    
    ADDITIONAL_USER_INFO = """Pour les regex concernant des noms propres, considérez les accents, tirets et apostrophes.
Pour les dates, tenez compte des formats internationaux (DD/MM/YYYY, MM/DD/YYYY, YYYY-MM-DD).
Pour les nombres, considérez les séparateurs différents selon les cultures (point, virgule).
Pensez à rendre les regex robustes face aux espaces superflus en début/fin.
Pour les formats d'identification, prenez en compte les variations de formatage (avec/sans espaces ou caractères spéciaux)."""
    
    process_regex_improvement(
        excel_file=EXCEL_FILE,
        output_file=OUTPUT_FILE,
        system_prompt=SYSTEM_PROMPT,
        additional_user_info=ADDITIONAL_USER_INFO,
        batch_size=5,
        save_interval=10
    )

Onglet regex_mapping transformé avec 35 lignes
Onglet regex_definition chargé avec 35 lignes
Jointure effectuée avec succès, résultat: 35 lignes


Traitement des lots:   0%|          | 0/7 [00:00<?, ?it/s]

Résultats intermédiaires sauvegardés dans regex_improved_results_temp_0.xlsx
Coût cumulatif: 0.053963 $


Traitement des lots: 100%|██████████| 7/7 [03:04<00:00, 26.32s/it]


Traitement terminé. Résultats sauvegardés dans regex_improved_results.xlsx
Coût total: 0.377563 $


#Correction déterministe avec fake ERROR llm

##Etape 1

In [0]:
import pandas as pd
import os
import json
from dataclasses import dataclass
import requests
from typing import Dict, List, Optional, Any
import time
from tqdm import tqdm

@dataclass
class AuthConfig:
    base_url: str
    client_id: str
    client_secret: str
    scope: str

class AuthClient:
    def __init__(self, config: AuthConfig):
        self.config = config

    def get_token(self) -> Dict[str, str]:
        try:
            response = requests.post(
                f"{self.config.base_url}/as/token.oauth2",
                headers={"Content-Type": "application/x-www-form-urlencoded"},
                data={
                    "grant_type": "client_credentials",
                    "client_id": self.config.client_id,
                    "client_secret": self.config.client_secret,
                    "scope": self.config.scope
                }
            )
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException as e:
            raise Exception(f"Échec de l'authentification: {str(e)}")

def get_auth_token() -> str:
    config = AuthConfig(
        base_url="https://onelogin.stg.axa.com",
        client_id="VJKkswZb",
        client_secret="yG3pl6nkYhvJlQVD1y7xbZBl692YRytdpUJeODKg5BAutoEgxhkqzXLFYGqH5zlp",
        scope="urn:grp:chatgpt"
    )
    client = AuthClient(config)
    result = client.get_token()
    return result.get("access_token", "")

class SecureGPTCorrectionsGenerator:
    def __init__(self, 
                token: str,
                temperature: float = 0.7,
                base_url: str = "https://api-pp.se.axa-go.axa.com/ago-m365-securegpt-bapi-v1-vrs",
                model: str = "gpt-4o-2024-08-06"):
        self.token = token
        self.temperature = temperature
        self.base_url = base_url
        self.model = model
        self.costs = {
            'input_token': 2.50/1_000_000,
            'output_token': 10.00/1_000_000
        }

    def calculate_cost(self, response: Dict) -> Dict[str, float]:
        metrics = {'token_cost': 0, 'total_cost': 0}

        if 'usage' in response:
            input_tokens = response['usage'].get('prompt_tokens', 0)
            output_tokens = response['usage'].get('completion_tokens', 0)
            metrics['token_cost'] = (input_tokens * self.costs['input_token'] + 
                                   output_tokens * self.costs['output_token'])
            metrics['total_cost'] = metrics['token_cost']

        return metrics

    def chat(self, messages: List[Dict[str, Any]], **kwargs) -> Dict:
        url = f"{self.base_url}/deployments/{self.model}/chat/completions"
        headers = {
            "Content-Type": "application/json",
            "Authorization": f"Bearer {self.token}"
        }
        data = {
            "messages": messages,
            "temperature": self.temperature,
            "max_tokens": kwargs.get("max_tokens", 4000)
        }

        try:
            response = requests.post(url, headers=headers, json=data)
            response.raise_for_status()
            api_response = response.json()
            cost_metrics = self.calculate_cost(api_response)
            api_response['cost_metrics'] = cost_metrics
            return api_response
        except requests.exceptions.RequestException as e:
            print(f"Détails de l'erreur API: {response.text if 'response' in locals() else 'Pas de réponse'}")
            raise Exception(f"Échec de l'appel API: {str(e)}")

    def generer_correction_manuelle(self, erreur, regex_info):
        """
        Génère une correction manuellement en cas d'échec du LLM
        """
        colonne = erreur['Columna']
        correction = ""

        # Logique basique basée sur le type de colonne
        if 'FECHA' in colonne:  # Date
            correction = "01/01/1980"
        elif 'NIF' in colonne or 'CIF' in colonne:  # Documents d'identité
            correction = "A12345678"
        elif 'NOMBRE' in colonne or 'APELLIDO' in colonne:  # Noms
            correction = "Exemple"
        elif 'NUM' in colonne or 'CUENTA' in colonne:  # Nombres
            correction = "1234567890"
        else:
            correction = "VALEUR_DEFAUT"

        print(f"Correction manuelle générée pour {colonne}: {correction}")
        return correction

    def generate_corrections_batch(self, 
                                  batch_data: List[Dict[str, Any]], 
                                  regex_data: Dict[str, Dict[str, Any]],
                                  system_prompt: str) -> List[Dict]:
        # Formatage des données pour le prompt
        formatted_batch = json.dumps(batch_data, ensure_ascii=False, indent=2)
        formatted_regex = json.dumps(regex_data, ensure_ascii=False, indent=2)
        
        user_content = f"""
Je dois corriger des erreurs dans un fichier d'assurance santé.

Voici les erreurs à corriger (le format est déjà défini, j'ai juste besoin de valeurs pour 'correction'):
{formatted_batch}

Voici les informations sur les formats attendus pour chaque colonne:
{formatted_regex}

Pour chaque ligne d'erreur, donnez-moi uniquement une valeur pour la colonne 'correction' qui:
1. Respecte le format (regex) de la colonne concernée
2. Est une donnée plausible pour un contexte d'assurance santé espagnol
3. Est variée (pas toujours la même valeur pour un même type de colonne)

Renvoyez exactement le même format JSON que celui que je vous ai fourni, mais avec la colonne 'correction' remplie.
"""
        
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_content}
        ]
        
        max_retries = 5
        retry_delay = 15
        
        for attempt in range(max_retries):
            try:
                response = self.chat(messages)
                response_content = response['choices'][0]['message']['content']
                
                # Log pour débug
                print(f"Réponse brute de l'API (premiers 200 caractères): {response_content[:200]}...")
                
                # Nettoyage de la réponse
                clean_content = response_content.strip()
                if clean_content.startswith("```json"):
                    clean_content = clean_content[7:]
                if clean_content.endswith("```"):
                    clean_content = clean_content[:-3]
                clean_content = clean_content.strip()
                
                try:
                    corrected_batch = json.loads(clean_content)
                    
                    # Vérifier que la réponse est valide
                    if not isinstance(corrected_batch, list):
                        raise ValueError("La réponse n'est pas une liste valide")
                    
                    for item in corrected_batch:
                        if not isinstance(item, dict) or 'correction' not in item:
                            raise ValueError("Un élément de la réponse est invalide ou manque la clé 'correction'")
                        item['cost_metrics'] = response.get('cost_metrics', {})
                    
                    return corrected_batch
                    
                except json.JSONDecodeERROR as e:
                    print(f"Erreur de décodage JSON: {str(e)}")
                    print(f"Contenu nettoyé: {clean_content[:200]}...")
                    raise
            
            except Exception as e:
                if attempt < max_retries - 1:
                    print(f"Erreur lors de la génération des corrections (tentative {attempt+1}/{max_retries}): {str(e)}")
                    print(f"Nouvelle tentative dans {retry_delay} secondes...")
                    time.sleep(retry_delay)
                else:
                    print(f"Échec de la génération après {max_retries} tentatives: {str(e)}")
                    # En cas d'échec après toutes les tentatives, générer manuellement
                    for item in batch_data:
                        colonne = item['Columna']
                        regex_info = regex_data.get(colonne, {})
                        item['correction'] = self.generer_correction_manuelle(item, regex_info)
                        item['cost_metrics'] = {'ERROR': str(e)}
                    return batch_data

def lire_fichiers(fichier_excel, fichier_regex):
    """
    Lit les onglets du fichier Excel et le fichier de regex
    """
    try:
        # Lire l'onglet 'Macros Alta' et 'ERROR'
        df_macros = pd.read_excel(fichier_excel, sheet_name='Macros Alta')
        df_ERROR = pd.read_excel(fichier_excel, sheet_name='ERROR')
        
        # Vérifier les noms de colonnes dans df_ERROR
        print(f"Colonnes dans l'onglet 'ERROR': {list(df_ERROR.columns)}")
        
        # S'assurer que les colonnes existent avec la bonne casse
        if 'Columna' not in df_ERROR.columns:
            if 'Columna' in df_ERROR.columns:
                df_ERROR.rename(columns={'Columna': 'Columna'}, inplace=True)
                print("Colonne 'Columna' renommée en 'Columna'")
            else:
                print("Attention: Colonne 'Columna' introuvable")
                
        if 'Linea' not in df_ERROR.columns:
            if 'Linea' in df_ERROR.columns:
                df_ERROR.rename(columns={'Linea': 'Linea'}, inplace=True)
                print("Colonne 'Linea' renommée en 'Linea'")
            else:
                print("Attention: Colonne 'Linea' introuvable")
        
        print(f"Onglet 'Macros Alta' chargé: {df_macros.shape[0]} lignes, {df_macros.shape[1]} colonnes")
        print(f"Onglet 'ERROR' chargé: {df_ERROR.shape[0]} lignes")
        
        # Lire le fichier regex amélioré
        df_regex = pd.read_excel(fichier_regex)
        print(f"Fichier regex chargé: {df_regex.shape[0]} règles")
        
        return df_macros, df_ERROR, df_regex
    except Exception as e:
        raise ValueError(f"Erreur lors de la lecture des fichiers: {str(e)}")

def preparer_regex_dict(df_regex):
    """
    Prépare un dictionnaire de regexes par colonne pour faciliter l'accès
    """
    regex_dict = {}
    for _, row in df_regex.iterrows():
        column_name = row['column_name']
        
        regex_dict[column_name] = {
            'regex_id': row['regex_id'] if 'regex_id' in row else None,
            'regex_rule': row['regex_rule'] if 'regex_rule' in row else "",
            'regex_description': row['regex_description'] if 'regex_description' in row else "",
            'regex_rule_amelio': row['regex_rule_amelio'] if 'regex_rule_amelio' in row and pd.notna(row['regex_rule_amelio']) else ""
        }
    
    return regex_dict

def preparer_donnees_erreur(df_ERROR):
    """
    Prépare les données d'erreur pour le LLM
    """
    # S'assurer que la colonne 'correction' existe
    if 'correction' not in df_ERROR.columns:
        df_ERROR['correction'] = ""
    
    # Convertir en liste de dictionnaires pour faciliter le traitement
    erreurs = []
    for _, row in df_ERROR.iterrows():
        erreur = {
            'Columna': row['Columna'],
            'Linea': row['Linea'],
            'correction': ""
        }
        erreurs.append(erreur)
    
    return erreurs

def generer_corrections_via_llm(erreurs, regex_dict, batch_size=3):
    """
    Génère des corrections via le LLM pour toutes les erreurs
    Réduit la taille des lots pour minimiser les erreurs
    """
    # Initialiser le générateur SecureGPT
    generator = SecureGPTCorrectionsGenerator(
        token=get_auth_token(),
        temperature=0.7
    )
    
    # Prompt système pour le LLM
    SYSTEM_PROMPT = """Vous êtes un expert en génération de données synthétiques pour des tests.
Votre tâche est de générer des valeurs réalistes pour corriger des erreurs dans un fichier de données d'assurance santé en Espagne.

Pour chaque ligne d'erreur que je vous donne, générez une valeur de correction qui:
1. Respecte strictement le format (regex) de la colonne concernée
2. Est une donnée synthétique mais plausible pour l'assurance santé
3. Est variée (évitez les répétitions)

Renvoyez uniquement un JSON valide avec le même format que l'entrée, mais avec la colonne "correction" remplie.
N'ajoutez aucune explication ou texte supplémentaire en dehors du JSON."""
    
    # Traiter par lots
    resultats = []
    total_cost = 0
    
    # Diviser en lots plus petits pour réduire les risques d'erreur
    for i in tqdm(range(0, len(erreurs), batch_size), desc="Traitement des lots"):
        batch = erreurs[i:i+batch_size]
        
        try:
            # Générer les corrections pour le lot
            corrected_batch = generator.generate_corrections_batch(
                batch, 
                regex_dict,
                SYSTEM_PROMPT
            )
            
            resultats.extend(corrected_batch)
            
            # Calculer le coût
            for item in corrected_batch:
                if 'cost_metrics' in item and 'total_cost' in item['cost_metrics']:
                    total_cost += item['cost_metrics']['total_cost']
            
            # Pause plus longue entre les lots
            time.sleep(2)
            
        except Exception as e:
            print(f"Erreur lors du traitement du lot {i//batch_size + 1}: {str(e)}")
            # En cas d'erreur, générer manuellement pour ce lot
            for item in batch:
                colonne = item['Columna']
                regex_info = regex_dict.get(colonne, {})
                item['correction'] = generator.generer_correction_manuelle(item, regex_info)
                resultats.append(item)
    
    print(f"Génération terminée. Coût total: {total_cost:.6f} $")
    return resultats

def mettre_a_jour_df_ERROR(df_ERROR, corrections):
    """
    Met à jour le DataFrame d'erreurs avec les corrections
    """
    # Créer un dictionnaire pour faciliter la recherche
    corrections_dict = {}
    for correction in corrections:
        # On utilise les noms de colonnes corrects (avec majuscules)
        key = (correction['Columna'], correction['Linea'])
        corrections_dict[key] = correction['correction']
    
    # Mettre à jour le DataFrame
    for index, row in df_ERROR.iterrows():
        key = (row['Columna'], row['Linea'])
        if key in corrections_dict:
            df_ERROR.at[index, 'correction'] = corrections_dict[key]
    
    return df_ERROR

# Fonction principale
def generer_corrections_llm(fichier_excel, fichier_regex, fichier_sortie):
    """
    Fonction principale pour générer les corrections via LLM
    """
    # Lire les fichiers
    df_macros, df_ERROR, df_regex = lire_fichiers(fichier_excel, fichier_regex)
    
    # Préparer le dictionnaire de regex
    regex_dict = preparer_regex_dict(df_regex)
    
    # Préparer les données d'erreur
    erreurs = preparer_donnees_erreur(df_ERROR)
    
    # Générer les corrections via LLM
    corrections = generer_corrections_via_llm(erreurs, regex_dict, batch_size=3)
    
    # Mettre à jour le DataFrame d'erreurs
    df_ERROR_corrige = mettre_a_jour_df_ERROR(df_ERROR, corrections)
    
    # Sauvegarder le résultat
    df_ERROR_corrige.to_excel(fichier_sortie, index=False)
    print(f"Corrections générées et sauvegardées dans {fichier_sortie}")

if __name__ == "__main__":
    # Fichiers d'entrée/sortie
    FICHIER_EXCEL = "250331200000003_Health_Listado_asegurados_anonym.xlsx"
    FICHIER_REGEX = "regex_improved_results.xlsx"
    FICHIER_SORTIE = "ERROR_avec_corrections_llm.xlsx"
    
    # Générer les corrections
    generer_corrections_llm(FICHIER_EXCEL, FICHIER_REGEX, FICHIER_SORTIE)

Colonnes dans l'onglet 'ERROR': ['Columna', 'Linea', 'Valor Incorrecto', 'Descripcion', 'Correction']
Onglet 'Macros Alta' chargé: 22 lignes, 35 colonnes
Onglet 'ERROR' chargé: 113 lignes
Fichier regex chargé: 35 règles


Traitement des lots:   0%|          | 0/38 [00:00<?, ?it/s]

Réponse brute de l'API (premiers 200 caractères): ```json
[
  {
    "Columna": "NUM_MIEMBROS_DE_LA_FAMILIA",
    "Linea": 0,
    "correction": "4"
  },
  {
    "Columna": "NUM_MIEMBROS_DE_LA_FAMILIA",
    "Linea": 4,
    "correction": "3"
  },
  {
  ...


Traitement des lots:   3%|▎         | 1/38 [00:09<06:02,  9.81s/it]

Réponse brute de l'API (premiers 200 caractères): ```json
[
  {
    "Columna": "NUM_MIEMBROS_DE_LA_FAMILIA",
    "Linea": 12,
    "correction": "3"
  },
  {
    "Columna": "NUM_MIEMBROS_DE_LA_FAMILIA",
    "Linea": 13,
    "correction": "5"
  },
  {
...


...

Réponse brute de l'API (premiers 200 caractères): ```json
[
  {
    "Columna": "FECHA_EFECTO",
    "Linea": 18,
    "correction": "15/03/2023"
  },
  {
    "Columna": "FECHA_EFECTO",
    "Linea": 19,
    "correction": "01/11/2022"
  },
  {
    "Colum...


Traitement des lots:  97%|█████████▋| 37/38 [03:51<00:06,  6.13s/it]

Réponse brute de l'API (premiers 200 caractères): ```json
[
  {
    "Columna": "FECHA_EFECTO",
    "Linea": 21,
    "correction": "15/04/2023"
  },
  {
    "Columna": "RAZON_SOCIAL",
    "Linea": 0,
    "correction": "Seguros Vida S.A."
  }
]
```...


Traitement des lots: 100%|██████████| 38/38 [03:57<00:00,  6.25s/it]


Génération terminée. Coût total: 1.279940 $
Corrections générées et sauvegardées dans ERROR_avec_corrections_llm.xlsx


##Etape2

In [0]:
import pandas as pd
from datetime import datetime

def lire_fichiers(fichier_data, fichier_corrections):
    try:
        # Lire l'onglet 'Macros Alta' du fichier principal
        df_macros = pd.read_excel(fichier_data, sheet_name='Macros Alta')
        
        # Lire le fichier de corrections
        df_corrections = pd.read_excel(fichier_corrections)
        
        print(f"Colonnes dans le fichier de corrections: {list(df_corrections.columns)}")
        
        # S'assurer que les colonnes nécessaires existent avec la bonne casse
        if 'Columna' not in df_corrections.columns and 'columna' in df_corrections.columns:
            df_corrections.rename(columns={'columna': 'Columna'}, inplace=True)
            print("Colonne 'columna' renommée en 'Columna'")
            
        if 'Linea' not in df_corrections.columns and 'linea' in df_corrections.columns:
            df_corrections.rename(columns={'linea': 'Linea'}, inplace=True)
            print("Colonne 'linea' renommée en 'Linea'")
        
        print(f"Données principales chargées: {df_macros.shape[0]} lignes, {df_macros.shape[1]} colonnes")
        print(f"Corrections chargées: {df_corrections.shape[0]} corrections")
        
        return df_macros, df_corrections
    except Exception as e:
        raise ValueError(f"Erreur lors de la lecture des fichiers: {str(e)}")

def appliquer_corrections(df_macros, df_corrections):
    # Vérifier que les colonnes nécessaires existent
    colonnes_requises = ['Columna', 'Linea', 'correction']
    for col in colonnes_requises:
        if col not in df_corrections.columns:
            raise ValueError(f"La colonne '{col}' est manquante dans le fichier de corrections")
    
    corrections_appliquees = 0
    corrections_ignorees = 0
    
    for idx, correction in df_corrections.iterrows():
        try:
            colonne = correction['Columna']
            
            # Conversion en entier avec gestion des erreurs
            try:
                ligne = int(float(correction['Linea']))
            except:
                print(f"Erreur de conversion pour ligne {correction['Linea']} (type: {type(correction['Linea'])})")
                ligne = -1
            
            nouvelle_valeur = correction['correction']
            
            if pd.isna(nouvelle_valeur) or nouvelle_valeur == "":
                print(f"Ignoré: Valeur de correction vide pour {colonne}[{ligne}]")
                corrections_ignorees += 1
                continue
            
            # Vérifier que la colonne existe
            if colonne not in df_macros.columns:
                print(f"Avertissement: Colonne '{colonne}' non trouvée dans les données principales")
                corrections_ignorees += 1
                continue
            
            # Vérifier que la ligne existe
            if ligne < 0 or ligne >= len(df_macros):
                print(f"Avertissement: Ligne {ligne} hors limites (0-{len(df_macros)-1})")
                corrections_ignorees += 1
                continue
                
            # Sauvegarder l'ancienne valeur pour le log
            ancienne_valeur = df_macros.at[ligne, colonne]
            
            # Traitement spécial pour les dates avec format DD/MM/YYYY
            if 'FECHA' in colonne and isinstance(nouvelle_valeur, str) and '/' in nouvelle_valeur:
                try:
                    # Convertir explicitement la date au format correct
                    date_parts = nouvelle_valeur.split('/')
                    if len(date_parts) == 3 and len(date_parts[0]) <= 2 and len(date_parts[1]) <= 2:
                        # C'est probablement une date DD/MM/YYYY
                        jour, mois, annee = map(int, date_parts)
                        nouvelle_valeur = f"{annee}-{mois:02d}-{jour:02d}"
                except:
                    # Si la conversion échoue, garder la valeur originale
                    pass
            
            # Appliquer la correction
            df_macros.at[ligne, colonne] = nouvelle_valeur
            
            print(f"Correction appliquée: {colonne}[{ligne}] = '{ancienne_valeur}' -> '{nouvelle_valeur}'")
            corrections_appliquees += 1
            
        except Exception as e:
            print(f"Erreur lors de l'application de la correction (index {idx}): {str(e)}")
            corrections_ignorees += 1
    
    print(f"\nRésumé: {corrections_appliquees} corrections appliquées, {corrections_ignorees} corrections ignorées")
    return df_macros

def appliquer_corrections_excel(fichier_data, fichier_corrections, fichier_sortie):
    # Charger les données
    df_macros, df_corrections = lire_fichiers(fichier_data, fichier_corrections)
    
    # Appliquer les corrections
    df_macros_corrige = appliquer_corrections(df_macros, df_corrections)
    
    # Sauvegarder le résultat
    with pd.ExcelWriter(fichier_sortie) as writer:
        df_macros_corrige.to_excel(writer, sheet_name='macros alta', index=False)
        # Conserver l'onglet 'ERROR' original
        pd.read_excel(fichier_data, sheet_name='ERROR').to_excel(writer, sheet_name='ERROR', index=False)
    
    print(f"\nDonnées corrigées sauvegardées dans {fichier_sortie}")

if __name__ == "__main__":
    # Fichiers d'entrée/sortie
    FICHIER_DATA = "250331200000003_Health_Listado_asegurados_anonym.xlsx"
    FICHIER_CORRECTIONS = "ERROR_avec_corrections_llm.xlsx"
    FICHIER_SORTIE = "250331200000003_Health_Listado_asegurados_corrige.xlsx"
    
    # Appliquer les corrections
    appliquer_corrections_excel(FICHIER_DATA, FICHIER_CORRECTIONS, FICHIER_SORTIE)

Colonnes dans le fichier de corrections: ['Columna', 'Linea', 'Valor Incorrecto', 'Descripcion', 'Correction', 'correction']
Données principales chargées: 22 lignes, 35 colonnes
Corrections chargées: 113 corrections
Correction appliquée: NUM_MIEMBROS_DE_LA_FAMILIA[0] = '4.0' -> '4'
Correction appliquée: NUM_MIEMBROS_DE_LA_FAMILIA[4] = '4.0' -> '3'
Correction appliquée: NUM_MIEMBROS_DE_LA_FAMILIA[8] = '3.0' -> '5'
Correction appliquée: NUM_MIEMBROS_DE_LA_FAMILIA[12] = '1.0' -> '3'
Correction appliquée: NUM_MIEMBROS_DE_LA_FAMILIA[13] = '1.0' -> '5'
Correction appliquée: NUM_MIEMBROS_DE_LA_FAMILIA[14] = '1.0' -> '2'
Correction appliquée: NUM_MIEMBROS_DE_LA_FAMILIA[15] = '1.0' -> '3'
Correction appliquée: NUM_MIEMBROS_DE_LA_FAMILIA[16] = '1.0' -> '4'
Correction appliquée: NUM_MIEMBROS_DE_LA_FAMILIA[17] = '1.0' -> '5'
Correction appliquée: NUM_MIEMBROS_DE_LA_FAMILIA[18] = '1.0' -> '3'
Correction appliquée: NUM_MIEMBROS_DE_LA_FAMILIA[19] = '1.0' -> '5'
Correction appliquée: NUM_MIEMBROS_DE_L